In [ ]:

#  This NoteBook contains code from the Ch 05 folder ..
#



# Step 00: Setup

In [1]:

%load_ext autoreload
%autoreload 2

%xmode Minimal

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate


   ###

    
#  How to use tabulate-
#
l_result = [{ "col1": 20, "col2": 30}]
   #
#  print(tabulate(l_result, headers='keys', tablefmt='psql', showindex=False))

print("--")


Exception reporting mode: Minimal


--


# Step 01: Iris Data

In [ ]:

   pd_nodes[l_each["label"]] = pd.read_csv( (l_folder + l_each["file"]), header = "infer", sep="|",
       dtype={"id": "string", "smiles": "string", "label": "string", "tcga_code": "string"})
   print("Number of records: %-8d   %-16s   %-32s" % (len(pd_nodes[l_each["label"]]), l_each["label"], l_each["file"]))
   print(tabulate(pd_nodes[l_each["label"]].head(2), headers='keys', tablefmt='psql', showindex=False))
   print()
